### Get the 12 most populated cities in Germany

##### Using City by API-Ninjas from rapidapi.com

In [2]:
from datetime import datetime, timedelta, timezone
from IPython.display import display, JSON
import pandas as pd
import requests

In [3]:

url = "https://city-by-api-ninjas.p.rapidapi.com/v1/city"

querystring = {"country":"DE","min_population":"500000","limit":"12"}

headers = {
	"X-RapidAPI-Host": "city-by-api-ninjas.p.rapidapi.com",
	"X-RapidAPI-Key": "**********************************"
}

response = requests.request("GET", url, headers=headers, params=querystring)
response_json = response.json()
display(JSON(response_json))

<IPython.core.display.JSON object>

In [4]:
df_cities_geo = pd.DataFrame(response_json)

In [5]:
df_cities_geo

,name,latitude,longitude,country,population,is_capital
0,Berlin,52.5167,13.3833,DE,3644826,True
1,Hamburg,53.5500,10.0000,DE,1841179,False
2,Munich,48.1372,11.5755,DE,1471508,False
3,Cologne,50.9422,6.9578,DE,1085664,False
4,Frankfurt,50.1136,8.6797,DE,753056,False
5,Bremen,53.1153,8.7975,DE,724909,False
6,Stuttgart,48.7761,9.1775,DE,634830,False
7,Dusseldorf,51.2311,6.7724,DE,619294,False
8,Dortmund,51.5139,7.4653,DE,587010,False
9,Essen,51.4508,7.0131,DE,583109,False


In [6]:
df_cities_geo = df_cities_geo.drop(["is_capital"], axis=1)

In [7]:
# Transform the population column to float

df_cities_geo['population'] = pd.to_numeric(df_cities_geo['population'], downcast='float')

In [117]:
# Create a new column which combines "city" and "country" columns.
# The values within the column are unique so can be used as a primery key later in the SQL.

df_cities_geo = (df_cities_geo.assign(municipality_iso_country = df_cities_geo["name"] + "," + df_cities_geo["country"]))
df_cities_geo = df_cities_geo.drop(["city_country"], axis=1)

In [92]:
df_cities_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          12 non-null     object 
 1   latitude      12 non-null     float64
 2   longitude     12 non-null     float64
 3   country       12 non-null     object 
 4   population    12 non-null     float32
 5   city_country  12 non-null     object 
dtypes: float32(1), float64(2), object(3)
memory usage: 656.0+ bytes


In [10]:
df_cities_geo

,name,latitude,longitude,country,population
0,Berlin,52.5167,13.3833,DE,3644826.0
1,Hamburg,53.5500,10.0000,DE,1841179.0
2,Munich,48.1372,11.5755,DE,1471508.0
3,Cologne,50.9422,6.9578,DE,1085664.0
4,Frankfurt,50.1136,8.6797,DE,753056.0
5,Bremen,53.1153,8.7975,DE,724909.0
6,Stuttgart,48.7761,9.1775,DE,634830.0
7,Dusseldorf,51.2311,6.7724,DE,619294.0
8,Dortmund,51.5139,7.4653,DE,587010.0
9,Essen,51.4508,7.0131,DE,583109.0


### Update data into database using sqlalchemy

#### Establish the connection

In [8]:
import sqlalchemy

In [85]:
schema="name of the schema"
host="provide the aws host"
user="root"
password="your password for MySQL"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

#### Import and update the tables to the SQL

In [122]:
df_cities_geo.to_sql('german_cities', con=con, if_exists='append', index=False)